In [1]:
# NF‑κB Luciferase Reporter Assay – pylabrobot version

import os
import sys
from pylabrobot.resources import Coordinate
from pylabrobot.liquid_handling.backends.chatterbox import LiquidHandlerChatterboxBackend
from pylabrobot.visualizer.visualizer import Visualizer
from pylabrobot.resources.opentrons import (
    OTDeck,
    corning_96_wellplate_360ul_flat,
    nest_12_reservoir_15ml,
    nest_1_reservoir_195ml,
    opentrons_96_tiprack_300ul
)
from pylabrobot.liquid_handling import LiquidHandler

In [2]:

# from pylabrobot.resources import set_volume_tracking
# from pylabrobot.resources import set_tip_tracking
# set_volume_tracking(enabled=True)
# set_tip_tracking(enabled=True)
# ──────────────────────────────────────
# User‑configurable constants (µL)
MEDIUM_VOL = 100     # volume of spent medium to remove
PBS_VOL    = 50      # PBS wash volume
LYSIS_VOL  = 30      # lysis buffer volume
LUC_VOL    = 100     # luciferase reagent volume
TOTAL_COL  = 12      # process A1–A12


# ──────────────────────────────────────

lh = LiquidHandler(backend=LiquidHandlerChatterboxBackend(), deck=OTDeck())
await lh.setup()
#vis = Visualizer(resource=lh)
#await vis.setup()

tiprack_slots = [1, 4, 8, 11]
tipracks = {
    f"tiprack_{slot}": opentrons_96_tiprack_300ul(name=f"tiprack_{slot}")
    for slot in tiprack_slots
}

for name, tiprack in tipracks.items():
    slot = int(name.split("_")[1])
    lh.deck.assign_child_at_slot(tiprack, slot=slot)

# Working 96‑well plate at slot 6
working_plate = corning_96_wellplate_360ul_flat(name="working_plate")
lh.deck.assign_child_at_slot(working_plate, slot=6)

# 12‑channel reservoir (PBS, Lysis, Luciferase) at slot 3
reagent_stock = nest_12_reservoir_15ml(name='reagent_stock')
lh.deck.assign_child_at_slot(reagent_stock, slot=3)

# 1‑channel waste reservoir at slot 9
waste_liq = nest_1_reservoir_195ml(name='waste_liq')
lh.deck.assign_child_at_slot(waste_liq, slot=9)

Setting up the liquid handler.
Resource deck was assigned to the liquid handler.
Resource trash_container was assigned to the liquid handler.
Resource tiprack_1 was assigned to the liquid handler.
Resource tiprack_4 was assigned to the liquid handler.
Resource tiprack_8 was assigned to the liquid handler.
Resource tiprack_11 was assigned to the liquid handler.
Resource working_plate was assigned to the liquid handler.
Resource reagent_stock was assigned to the liquid handler.
Resource waste_liq was assigned to the liquid handler.


In [3]:
pbs        = reagent_stock[0][0]
lysis      = reagent_stock[1][0]
luciferase = reagent_stock[2][0]
waste_liq  = waste_liq[0]
wells_name = [f"A{i}" for i in range(1, 13)]
cells_all  = working_plate[wells_name] # A1–A12

In [4]:
working_plate_volumes = [
    ('culture medium', MEDIUM_VOL) if i % 8 == 0 else (None, 0)
    for i in range(96)
]
working_plate.set_well_liquids(working_plate_volumes)
reagent_info = [('PBS Buffer', 5000), ('Lysis Buffer', 5000), ('Luciferase Reagent', 5000)]+[ (None, 0) ]* 9
reagent_stock.set_well_liquids(reagent_info)
# lh.deck.set_tiprack(list(tipracks.values()))

In [5]:
# state = reagent_stock.serialize_all_state()
# print(state)
reagent_stock.load_all_state({'reagent_stock': {}, 'reagent_stock_A1': {'liquids': [['PBS Buffer', 5000]], 'pending_liquids': [], 'liquid_history': []}, 'reagent_stock_A2': {'liquids': [['Lysis Buffer', 5000]], 'pending_liquids': [['Lysis Buffer', 5000]], 'liquid_history': ['Lysis Buffer']}, 'reagent_stock_A3': {'liquids': [['Luciferase Reagent', 5000]], 'pending_liquids': [['Luciferase Reagent', 5000]], 'liquid_history': ['Luciferase Reagent']}, 'reagent_stock_A4': {'liquids': [[None, 0]], 'pending_liquids': [[None, 0]], 'liquid_history': [None]}, 'reagent_stock_A5': {'liquids': [[None, 0]], 'pending_liquids': [[None, 0]], 'liquid_history': [None]}, 'reagent_stock_A6': {'liquids': [[None, 0]], 'pending_liquids': [[None, 0]], 'liquid_history': [None]}, 'reagent_stock_A7': {'liquids': [[None, 0]], 'pending_liquids': [[None, 0]], 'liquid_history': [None]}, 'reagent_stock_A8': {'liquids': [[None, 0]], 'pending_liquids': [[None, 0]], 'liquid_history': [None]}, 'reagent_stock_A9': {'liquids': [[None, 0]], 'pending_liquids': [[None, 0]], 'liquid_history': [None]}, 'reagent_stock_A10': {'liquids': [[None, 0]], 'pending_liquids': [[None, 0]], 'liquid_history': [None]}, 'reagent_stock_A11': {'liquids': [[None, 0]], 'pending_liquids': [[None, 0]], 'liquid_history': [None]}, 'reagent_stock_A12': {'liquids': [[None, 0]], 'pending_liquids': [[None, 0]], 'liquid_history': [None]}})

reagent_stock[0][0].tracker.liquids

[('PBS Buffer', 5000)]

In [12]:
from pylabrobot.resources import set_tip_tracking, set_volume_tracking
set_tip_tracking(True), set_volume_tracking(True)

(None, None)

In [6]:
lh.serialize()

{'name': 'lh_deck',
 'type': 'LiquidHandler',
 'size_x': 624.3,
 'size_y': 565.2,
 'size_z': 900,
 'location': {'x': 0, 'y': 0, 'z': 0, 'type': 'Coordinate'},
 'rotation': {'x': 0, 'y': 0, 'z': 0, 'type': 'Rotation'},
 'category': 'liquid_handler',
 'model': None,
 'children': [{'name': 'deck',
   'type': 'OTDeck',
   'size_x': 624.3,
   'size_y': 565.2,
   'size_z': 900,
   'location': {'x': 0, 'y': 0, 'z': 0, 'type': 'Coordinate'},
   'rotation': {'x': 0, 'y': 0, 'z': 0, 'type': 'Rotation'},
   'category': 'deck',
   'children': [{'name': 'trash_container',
     'type': 'Resource',
     'size_x': 172.86,
     'size_y': 165.86,
     'size_z': 82,
     'location': {'x': 265.0, 'y': 271.5, 'z': 0.0, 'type': 'Coordinate'},
     'rotation': {'x': 0, 'y': 0, 'z': 0, 'type': 'Rotation'},
     'category': None,
     'model': None,
     'children': [{'name': 'trash',
       'type': 'Trash',
       'size_x': 172.86,
       'size_y': 165.86,
       'size_z': 82,
       'location': {'x': 0, 'y':

In [6]:
await lh.remove_liquid(
    vols=[MEDIUM_VOL]*12,
    sources=cells_all,
    waste_liquid=waste_liq,
    top=[-0.2],
    liquid_height=[0.2,0],
    flow_rates=[0.2,3],
    offsets=[Coordinate(-2.5, 0, 0),Coordinate(0, 0, -5)]
)

AttributeError: 'LiquidHandler' object has no attribute 'remove_liquid'

In [15]:
await lh.remove_liquid(
    vols=[PBS_VOL*1.5]*12,
    top=[-0.2],
    liquid_height=[0.2,None],
    offsets=[Coordinate(-2.5,0,0),Coordinate(0,0,-5)],
    flow_rates=[0.2,3],
    sources=cells_all,
    waste_liquid=waste_liq
)

Picking up tips:
pip#  resource             offset           tip type     max volume (µL)  fitting depth (mm)   tip length (mm)  filter    
  p0: tiprack_1_E2         0,0,0            Tip          300.0            7.47                 59.3             No        
Tracker only has 0uL, please pay attention.
Container has too little liquid: 75.0uL > 0uL, please pay attention.
Air bubble detected, please pay attention.
Aspirating:
pip#  vol(ul)  resource             offset           flow rate  blowout    lld_z       
  p0: 75.0     working_plate_A1     -2.5,0,0         0.2        None       0.2        
[Well(name=waste_liq_A1, location=Coordinate(010.480, 007.140, 004.550), size_x=106.8, size_y=71.2, size_z=25, category=well)]
Tracker only has 0uL, please pay attention.
Container has too little liquid: 75.0uL > 0uL, please pay attention.
Air bubble detected, please pay attention.
Dispensing:
pip#  vol(ul)  resource             offset           flow rate  blowout    lld_z       
  p0: 75.0 

In [16]:
await lh.add_liquid(
    asp_vols=[LYSIS_VOL]*12,
    dis_vols=[LYSIS_VOL]*12,
    reagent_sources=[lysis],
    targets=cells_all,
    flow_rates=[0.5,0.3],
    liquid_height=[0.5,5],
    delays=[2,2]
)

Picking up tips:
pip#  resource             offset           tip type     max volume (µL)  fitting depth (mm)   tip length (mm)  filter    
  p0: tiprack_1_A4         0,0,0            Tip          300.0            7.47                 59.3             No        
Aspirating:
pip#  vol(ul)  resource             offset           flow rate  blowout    lld_z       
  p0: 30.0     reagent_stock_A2     0,0,0            0.5        None       0.5        
[Well(name=working_plate_A1, location=Coordinate(011.954, 071.814, 003.550), size_x=4.851, size_y=4.851, size_z=10.67, category=well)]
Dispensing:
pip#  vol(ul)  resource             offset           flow rate  blowout    lld_z       
  p0: 30.0     working_plate_A1     0,0,0            0.3        None       5.0        
Aspirating:
pip#  vol(ul)  resource             offset           flow rate  blowout    lld_z       
  p0: 0.0      working_plate_A1     -2.4,0,0         None       None       None       
Aspirating:
pip#  vol(ul)  resource      

In [17]:
lh.custom_delay(180)

<coroutine object DPLiquidHandler.custom_delay at 0x122e506d0>

In [18]:
await lh.add_liquid(
    asp_vols=[LUC_VOL]*12,
    dis_vols=[LUC_VOL+20]*12,
    reagent_sources=[luciferase],
    targets=cells_all,
    liquid_height=[0.5, None],
    mix_time=3,
    mix_vol=75,
    mix_rate=3,
    mix_liquid_height=0.5,
    delays = [2, None],
    blow_out_air_volume=[20,None],
    flow_rates=[0.75,0.75]
)

Picking up tips:
pip#  resource             offset           tip type     max volume (µL)  fitting depth (mm)   tip length (mm)  filter    
  p0: tiprack_1_B4         0,0,0            Tip          300.0            7.47                 59.3             No        
Aspirating:
pip#  vol(ul)  resource             offset           flow rate  blowout    lld_z       
  p0: 100.0    reagent_stock_A3     0,0,0            0.75       20.0       0.5        
[Well(name=working_plate_A1, location=Coordinate(011.954, 071.814, 003.550), size_x=4.851, size_y=4.851, size_z=10.67, category=well)]
Tracker only has 100.0uL, please pay attention.
Container has too little liquid: 120.0uL > 100.0uL, please pay attention.
Air bubble detected, please pay attention.
Dispensing:
pip#  vol(ul)  resource             offset           flow rate  blowout    lld_z       
  p0: 120.0    working_plate_A1     0,0,0            0.75       None       None       
Aspirating:
pip#  vol(ul)  resource             offset         